In [2]:
from pystac_client import Client
import stackstac
import rioxarray as rxr
import xarray as xr
import re

In [3]:


def mosaic_sentinel2_from_stac(stac_url: str, product_id: str, bands=("B04", "B03", "B02"), nodata_value=0):
    """
    Creates an imagery mosaic from Sentinel-2 STAC using a given product_id (e.g., 'S2C_MSIL2A_20250416T162921_N0511_R083').

    Parameters:
        stac_url (str): URL to the STAC API or static catalog.
        product_id (str): Sentinel-2 product ID to filter on.
        bands (tuple): Bands to include in the mosaic (default is RGB).
        nodata_value (int/float): Nodata value to apply for the mosaic.

    Returns:
        xarray.DataArray: Mosaic of the specified bands.
    """

    # Parse relevant values from product ID
    match = re.match(r"S2[CAB]_MSIL2A_(\d{8})T\d+_N\d+_R(\d+)", product_id)
    if not match:
        raise ValueError("Product ID format not recognized.")
    date_str, rel_orbit = match.groups()
    date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"

    # Connect to STAC
    catalog = Client.open(stac_url)

    # Search for matching items
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        query={"s2:datatake_id": {"eq": product_id}},
        datetime=f"{date}T00:00:00Z/{date}T23:59:59Z",
        limit=100
    )

    items = list(search.items())
    if not items:
        raise ValueError(f"No items found for product_id: {product_id}")

    # Load the mosaic using stackstac
    mosaic = stackstac.stack(
        items,
        assets=list(bands),
        epsg=items[0].properties.get("proj:epsg"),
        chunksize=2048,
        resolution=10,
        bounds_latlon=None,
        dtype="uint16",
        fill_value=nodata_value,
    )

    # Mosaic along time axis (in case of overlap)
    mosaic = mosaic.max("time")

    return mosaic


In [4]:
mosaic = mosaic_sentinel2_from_stac(
    stac_url="https://earth-search.aws.element84.com/v1",
    product_id="S2C_MSIL2A_20250416T162921_N0511_R083"
)



ValueError: No items found for product_id: S2C_MSIL2A_20250416T162921_N0511_R083

In [ ]:
# Plot RGB
rgb = mosaic.sel(band=["B04", "B03", "B02"]).transpose("band", "y", "x")
rgb_plot = (rgb / 3000).clip(0, 1)  # Simple stretch
rgb_plot.plot.imshow(figsize=(10, 10))